In [23]:
!pip install transformers
!pip


[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [25]:
import torch
import os
import pandas as pd
import re
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import classification_report
# from transformers.tokenization_bert_japanese import BertJapaneseTokenizer

- `torch`: PyTorchライブラリをインポートします. これは, 深層学習モデルを構築するための主要なライブラリです. 
- `transformers`: Transformerライブラリをインポートします. このライブラリは, BERTやGPT-2などの最先端自然言語処理モデルを簡単に利用できるようにします. 
- `AdamW`: AdamWオプティマイザをインポートします. これは, 深層学習モデルの訓練に使用される手法です. 
- `get_linear_schedule_with_warmup`: 学習率を徐々に上げていくスケジュールを取得するための関数をインポートします. 
- `DataLoader`: PyTorch DataLoaderをインポートします. これは, データセットを効率的に読み込むためのツールです. 
- `Dataset`: PyTorch Datasetクラスをインポートします. これは, カスタムデータセットを作成するための基底クラスです. 
- `pandas`: データ分析ライブラリであるpandasをインポートします. 
- `re`: 正規表現ライブラリであるreをインポートします. 

In [7]:
# 口コミデータ
reviews = [
    "鮨 さいとうに4回目の訪問. 今回はランチでディナー同様, 摘みありのコースを注文させて頂きました. タイトルにもありますが, 気付けば2年ぶりの訪問. 食べ歩きをしてると2.3年再訪してないお店が沢山出てきますが, 再訪してもそれほど月日が経ってないように感じるのは私だけでしょうか. . 笑さて, 本題のお料理ですが, 摘み握り共に何を食べても美味しく, 今回も流石の安定感です. 何一つとして気になるものはありません. また, この日は以前よりずっと飲んでみたかった黒龍酒造の『ESHIKOTO』を頂きました. 超入手困難なスパークリングの日本酒です. 当たりがとてもドライで食事の邪魔を一切しないテイスト. 香りはほのかに日本酒を感じさせます. こんな貴重なお酒を入るのはさすが齋藤さんですね. 次回はいつ来れるか分かりませんが, またぜひ伺いたいです. ご馳走様でした. ",
    "個人的に, 都内では大好きなお店の一つ. 3度目の訪問ですが, さいとうさんは時期が変わっても毎度ツマミも握りも安定感があり素晴らしいですね. この日, 初めて頂く「穴子の白焼き」は今まで食べた事ないくらい身がぷりぷりしており, 特に印象深い一品でした. さらに1人30,000円ちょっとと, 内容対してコスパの高さも嬉しいですね. また, 昼でも夜でも時期を変えて伺いたいです. ご馳走様でした. ",
    "今月2度目のさいとうさんへ. 今回はランチ利用です. 握り15貫のみのコースを頂きました. （人によっては少し量が少なく感じる方もいらっしゃるかもしれません. ）マグロ中心に今回も大変美味しく頂きました. 価格も1人16,500円と, とても良心的な価格でびっくりしました. この立地でこの価格だと, そりゃ中々予約取れないわかです. また伺います. ご馳走様でした. ",
    "言わずと知れた4年連続食べログゴールド, 2019年まで10年連続ミシュラン3つ星のお店です. 今回はお誘い頂き訪問してきました. その人気ぶりから現在では, 完全会員制を取っており, 一見での予約は不可になっております. さて, お寿司に関しては非常に素晴らしいです. できることなら会員になって通い続けたいお店です. ネタもシャリもとても個人的に好みでした. 特にシャリは酢と塩味が強く無く, 割と万人受けするかな, といった印象. 次いついけるか分かりませんが, ぜひまた伺いたいです. ご馳走様でした. ",
    "楽しい時間を過ごす事が出来ました. また是非伺いたいと思います. ご馳走様でした. ",
    "今回は個室で『鮨さいとう』二番手の沼尾さんの鮨を堪能しました. 仕事も所作も丁寧で, 安心して食に集中出来ました. 酢飯の酸度・塩味・硬さ等かなり好み. ネタも間違いのないものばかりでした！ご馳走様でした. ",
    "最高峰と言われるお鮨屋さんお鮨だけを楽しみに行くのではなく, 大将との会話, 雰囲気, 異空間を味わいながら食べるお鮨屋さん. お鮨は美味しい. 特にこれがというのはないが, 美味しい. 1番驚いたのは鮨屋とは思えない独特な雰囲気があるお店. 大将とお客さんの一体感のある空間でお鮨を楽しめます. 大将の人柄で人気なっているお店. お鮨だけを食べに行くには勿体無いいいお鮨屋さんでした. さいとうさんのにぎるお鮨を食べられたことに感謝. ",
    "念願の大将のお席!カツオのたたきはこれまで食べた1番の美味しさ. 熟成されたねっとり感で, 旨みがギュッ!パサつき全く無い仕上がり. 優しい, 鮑, たこ, など, おつまみも最高!握りはふわっと優しいシャリとの一体感が素晴らしい. 最初のあたたかめのシャリと白身が新鮮で美味. 甘すぎないツメもよく, 蛤や穴子も, 素材の美味しさか際立つ！来年ですが, また, 次回も楽しみ！",
    "日本の鮨店, 最高峰の1つ. 超予約困難店です. 1度訪れてみたかったお店ですが, 常連様にお誘い頂き, 伺うことができました. こちらのお店は・10年連続ミシュラン3つ星も会員制にするために返上・食べログGOLD(全国で30店舗ほどのみ)連続獲得・世界のグルメガイド, フランスの「ラ・リスト2024」で99.5点の日本最高評価を獲得. また, 世界でも上位7位. ・新規予約は基本受付しておらず, 食オクでのオークションのみ・食オクでは1席40万円や70万円の値がつくこともお店は六本木一丁目駅直結のアークヒルズタワー1階にあります. やや分かりづらい場所です. 店内は個室が2つあり, 個室内はL字のカウンター8席ほど. この日は, 2番手の沼尾大将の貸切会でした. お鮨はもちろんですが蒸し鮑と蛸の桜煮の旨味が凄く, 鰹の皮目がパリッとした火入れと最初のつまみがとても美味しく感動しました. つまみも握りもハイクオリティですが会話も楽しく雰囲気づくりも抜群. 価格は高いですが, 他のお鮨やさんと一線を画すほど美味しく満足度は高いです. 純粋に美味しくて, また来たいなぁと思える素晴らしいお鮨でした！ご馳走様でした！",
    "日本最高峰のお鮨のお店に, 幸運にもお誘いいただき, 訪問することができました. この日は, 個室の沼尾大将に握っていただきました. 貸切ということもあり, リラックスして, 美味しいお酒を飲みながら, 最高のお鮨を食べて, 幸せな時間を過ごすことができました. ご馳走様でした！！！",
    "食べログお鮨ランキング1位にしてミシュラン3つ星, 泣く子もだまるさいとう, 2月に続いて3月もさいとうさんへ, ミシュラン3つ星を取ったのは随分前だが最近になっても美味しさが増している気がする. ",
    "日本最高峰のお寿司と聞いていたのですが, お値段に比べてあまり美味しいと思いませんでした. ",
    "イベントの際にお邪魔しました. 少し値段が高い気がします. ",
    "イカやタコが美味しかったですが, マグロは微妙でした.",
    "お値段が高いので, もう少し美味しいものが食べたかったです.",
    "雰囲気が少し怖く感じました.",
    "味がイマイチだったので, また行くかは検討中です.",
    "お値段が高いので, 期待していたほど美味しくなかったです.",
    "電話対応が悪かったので, また行くかは検討中です.",
]

# 感情ラベルの追加（手動で設定, ここでは全てポジティブと仮定）
# labels = [1] * len(reviews)


# 感情ラベルの追加（ポジティブ: 1, ネガティブ: 0）
labels = [1] * 11 + [0] * 8

output_dir = 'text'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for i, (review, label) in enumerate(zip(reviews, labels)):
    df = pd.DataFrame({'text': [review], 'label': [label]})
    df.to_csv(os.path.join(output_dir, f'review_{i}.csv'), index=False)


**1. 口コミデータの準備**

```python
reviews = [
    "鮨 さいとうに4回目の訪問. 今回はランチでディナー同様, 摘みありのコースを注文させて頂きました. タイトルにもありますが, 気付けば2年ぶりの訪問. 食べ歩きをしてると2.3年再訪してないお店が沢山出てきますが, 再訪してもそれほど月日が経ってないように感じるのは私だけでしょうか. . 笑さて, 本題のお料理ですが, 摘み握り共に何を食べても美味しく, 今回も流石の安定感です. 何一つとして気になるものはありません. また, この日は以前よりずっと飲んでみたかった黒龍酒造の『ESHIKOTO』を頂きました. 超入手困難なスパークリングの日本酒です. 当たりがとてもドライで食事の邪魔を一切しないテイスト. 香りはほのかに日本酒を感じさせます. こんな貴重なお酒を入るのはさすが齋藤さんですね. 次回はいつ来れるか分かりませんが, またぜひ伺いたいです. ご馳走様でした. ",
    # ... (9件の口コミデータ省略)
]

labels = [1] * len(reviews)
```

- `reviews`: リスト形式で, 12個の「鮨 さいとう」の口コミデータが格納されています. 
- `labels`: 各口コミデータに対応する感情ラベルをリスト形式で格納しています. **ここでは全て1（ポジティブ）に設定**されています. 

**2. 個別のCSVファイルへの保存**

```python
for i, (review, label) in enumerate(zip(reviews, labels)):
    df = pd.DataFrame({'text': [review], 'label': [label]})
    df.to_csv(f'review_{i}.csv', index=False)
```

- 上記の口コミデータと感情ラベルを, `review_0.csv`, `review_1.csv` ... のように個別のCSVファイルに保存します. 
    - `pd.DataFrame`: 口コミデータと感情ラベルをDataFrameに変換します. 
    - `to_csv`: DataFrameをCSVファイルに出力します. 



In [8]:
# データフレームの作成
all_data = pd.DataFrame({'text': reviews, 'label': labels})

# 列名の変更
all_data.rename(columns={'text': '口コミ内容', 'label': '評価'}, inplace=True)

# インデックスを列に格納
all_data.reset_index(drop=True, inplace=True)

# フレーム作成
frame = all_data

# 左揃え設定
frame.style.set_properties(**{'text-align': 'left'})

,口コミ内容,評価
0,"鮨 さいとうに4回目の訪問. 今回はランチでディナー同様, 摘みありのコースを注文させて頂きました. タイトルにもありますが, 気付けば2年ぶりの訪問. 食べ歩きをしてると2.3年再訪してないお店が沢山出てきますが, 再訪してもそれほど月日が経ってないように感じるのは私だけでしょうか. . 笑さて, 本題のお料理ですが, 摘み握り共に何を食べても美味しく, 今回も流石の安定感です. 何一つとして気になるものはありません. また, この日は以前よりずっと飲んでみたかった黒龍酒造の『ESHIKOTO』を頂きました. 超入手困難なスパークリングの日本酒です. 当たりがとてもドライで食事の邪魔を一切しないテイスト. 香りはほのかに日本酒を感じさせます. こんな貴重なお酒を入るのはさすが齋藤さんですね. 次回はいつ来れるか分かりませんが, またぜひ伺いたいです. ご馳走様でした.",1
1,"個人的に, 都内では大好きなお店の一つ. 3度目の訪問ですが, さいとうさんは時期が変わっても毎度ツマミも握りも安定感があり素晴らしいですね. この日, 初めて頂く「穴子の白焼き」は今まで食べた事ないくらい身がぷりぷりしており, 特に印象深い一品でした. さらに1人30,000円ちょっとと, 内容対してコスパの高さも嬉しいですね. また, 昼でも夜でも時期を変えて伺いたいです. ご馳走様でした.",1
2,"今月2度目のさいとうさんへ. 今回はランチ利用です. 握り15貫のみのコースを頂きました. （人によっては少し量が少なく感じる方もいらっしゃるかもしれません. ）マグロ中心に今回も大変美味しく頂きました. 価格も1人16,500円と, とても良心的な価格でびっくりしました. この立地でこの価格だと, そりゃ中々予約取れないわかです. また伺います. ご馳走様でした.",1
3,"言わずと知れた4年連続食べログゴールド, 2019年まで10年連続ミシュラン3つ星のお店です. 今回はお誘い頂き訪問してきました. その人気ぶりから現在では, 完全会員制を取っており, 一見での予約は不可になっております. さて, お寿司に関しては非常に素晴らしいです. できることなら会員になって通い続けたいお店です. ネタもシャリもとても個人的に好みでした. 特にシャリは酢と塩味が強く無く, 割と万人受けするかな, といった印象. 次いついけるか分かりませんが, ぜひまた伺いたいです. ご馳走様でした.",1
4,楽しい時間を過ごす事が出来ました. また是非伺いたいと思います. ご馳走様でした.,1
5,"今回は個室で『鮨さいとう』二番手の沼尾さんの鮨を堪能しました. 仕事も所作も丁寧で, 安心して食に集中出来ました. 酢飯の酸度・塩味・硬さ等かなり好み. ネタも間違いのないものばかりでした！ご馳走様でした.",1
6,"最高峰と言われるお鮨屋さんお鮨だけを楽しみに行くのではなく, 大将との会話, 雰囲気, 異空間を味わいながら食べるお鮨屋さん. お鮨は美味しい. 特にこれがというのはないが, 美味しい. 1番驚いたのは鮨屋とは思えない独特な雰囲気があるお店. 大将とお客さんの一体感のある空間でお鮨を楽しめます. 大将の人柄で人気なっているお店. お鮨だけを食べに行くには勿体無いいいお鮨屋さんでした. さいとうさんのにぎるお鮨を食べられたことに感謝.",1
7,"念願の大将のお席!カツオのたたきはこれまで食べた1番の美味しさ. 熟成されたねっとり感で, 旨みがギュッ!パサつき全く無い仕上がり. 優しい, 鮑, たこ, など, おつまみも最高!握りはふわっと優しいシャリとの一体感が素晴らしい. 最初のあたたかめのシャリと白身が新鮮で美味. 甘すぎないツメもよく, 蛤や穴子も, 素材の美味しさか際立つ！来年ですが, また, 次回も楽しみ！",1
8,"日本の鮨店, 最高峰の1つ. 超予約困難店です. 1度訪れてみたかったお店ですが, 常連様にお誘い頂き, 伺うことができました. こちらのお店は・10年連続ミシュラン3つ星も会員制にするために返上・食べログGOLD(全国で30店舗ほどのみ)連続獲得・世界のグルメガイド, フランスの「ラ・リスト2024」で99.5点の日本最高評価を獲得. また, 世界でも上位7位. ・新規予約は基本受付しておらず, 食オクでのオークションのみ・食オクでは1席40万円や70万円の値がつくこともお店は六本木一丁目駅直結のアークヒルズタワー1階にあります. やや分かりづらい場所です. 店内は個室が2つあり, 個室内はL字のカウンター8席ほど. この日は, 2番手の沼尾大将の貸切会でした. お鮨はもちろんですが蒸し鮑と蛸の桜煮の旨味が凄く, 鰹の皮目がパリッとした火入れと最初のつまみがとても美味しく感動しました. つまみも握りもハイクオリティですが会話も楽しく雰囲気づくりも抜群. 価格は高いですが, 他のお鮨やさんと一線を画すほど美味しく満足度は高いです. 純粋に美味しくて, また来たいなぁと思える素晴らしいお鮨でした！ご馳走様でした！",1
9,"日本最高峰のお鮨のお店に, 幸運にもお誘いいただき, 訪問することができました. この日は, 個室の沼尾大将に握っていただきました. 貸切ということもあり, リラックスして, 美味しいお酒を飲みながら, 最高のお鮨を食べて, 幸せな時間を過ごすことができました. ご馳走様でした！！！",1


**1. データフレームの作成**

```python
all_data = pd.DataFrame({'text': reviews, 'label': labels})
```

- `pd.DataFrame`: リスト形式で格納された口コミデータ (`reviews`) と感情ラベル (`labels`) を, PandasのDataFrame形式に変換します. 
- `'text'`: 口コミ内容を格納する列を `'text'` と命名します. 
- `'label'`: 感情ラベルを格納する列を `'評価'` と命名します. 

**2. 列名の変更**

```python
all_data.rename(columns={'text': '口コミ内容', 'label': '評価'}, inplace=True)
```

- `rename(columns=...)`: 列名を変更します. 
    - `'text'`: 元の列名
    - `'口コミ内容'`: 変更後の列名
    - `inplace=True`: 変更を元のDataFrameに反映します. 
- 同様に, `'label'` 列名を `'評価'` に変更します. 

**3. インデックスを列に格納**

```python
all_data.reset_index(drop=True, inplace=True)
```

- `reset_index`: DataFrameの行インデックスを列として追加します. 
    - `drop=True`: 元の行インデックスを削除します. 
    - `inplace=True`: 変更を元のDataFrameに反映します. 

**4. フレーム作成**

```python
frame = all_data
```

- `frame = all_data`: 上記で加工したDataFrameを `frame` という変数に格納します. 

**5. 左揃え設定**

```python
frame.style.set_properties(**{'text-align': 'left'})
```

- `style.set_properties`: DataFrameの表示形式を設定します. 
    - `**{...}``: 辞書形式で, 設定したいプロパティとその値を指定します. 
    - `'text-align': 'left'`: '口コミ内容' 列の文字列を左揃えに設定します. 

In [9]:
# データセットの作成
class ReviewDataset(Dataset):
    def __init__(self, reviews, labels, tokenizer, max_len):
        self.reviews = reviews
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

**1. クラスの定義**

```python
class ReviewDataset(Dataset):
```

- `class ReviewDataset(Dataset)`: `Dataset` クラスを継承した `ReviewDataset` クラスを定義します. 
- `Dataset` クラスは, PyTorchでデータローダーと連携してデータ処理を行うための基底クラスです. 

**2. コンストラクタ (`__init__`)**

```python
def __init__(self, reviews, labels, tokenizer, max_len):
    self.reviews = reviews
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len
```

- `__init__(self, reviews, labels, tokenizer, max_len)`: コンストラクタは, データセットを作成する際に呼び出されるメソッドです. 
    - `reviews`: 口コミデータのリスト
    - `labels`: 感情ラベルのリスト
    - `tokenizer`: Transformerライブラリのトークナイザ
    - `max_len`: 最大トークン長
- 上記の引数を, クラスのインスタンス変数として保持します. 

**3. データ件数取得 (`__len__`)**

```python
def __len__(self):
    return len(self.reviews)
```

- `__len__(self)`: データセットに含まれるデータ件数を返します. 
- `len(self.reviews)`: `reviews` リストの長さを返します. 

**4. データ取得 (`__getitem__`)**

```python
def __getitem__(self, item):
    review = str(self.reviews[item])
    label = self.labels[item]

    encoding = self.tokenizer.encode_plus(
        review,
        add_special_tokens=True,
        max_length=self.max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )

    return {
        'review_text': review,
        'input_ids': encoding['input_ids'].flatten(),
        'attention_mask': encoding['attention_mask'].flatten(),
        'label': torch.tensor(label, dtype=torch.long)
    }
```

- `__getitem__(self, item)`: 特定のインデックス `item` に対応するデータを返します. 
    - `review`: `reviews` リストから `item` 番目の口コミデータを取得します. 
    - `label`: `labels` リストから `item` 番目の感情ラベルを取得します. 
    - `tokenizer.encode_plus`: Transformerライブラリの `encode_plus` 関数を使って, 口コミデータをモデルに入力できる形式に変換します. 
        - `review`: 変換する口コミデータ
        - `add_special_tokens=True`: 特別トークンを追加します. 
        - `max_length=self.max_len`: 最大トークン長を `self.max_len` に設定します. 
        - `return_token_type_ids=False`: トークンタイプIDを返しません. 
        - `padding='max_length'`: 最大トークン長までパディングを行います. 
        - `return_attention_mask=True`: アテンションマスクを返します. 
        - `return_tensors='pt'`: PyTorchテンソルとして返します. 
    - 変換されたデータと感情ラベルを辞書形式でまとめ, 返します. 
        - `'review_text'`: 変換前の口コミデータ
        - `'input_ids'`: トークンIDのテンソル
        - `'attention_mask'`: アテンションマスクのテンソル
        - `'label'`: 感情ラベルのテンソル（long型）

In [10]:
# データの読み込みと前処理
def load_and_preprocess_data(filepaths):
    reviews = []
    labels = []
    
    for filepath in filepaths:
        df = pd.read_csv(filepath)
        reviews.extend(df['text'].tolist())
        labels.extend(df['label'].tolist())

    return reviews, labels

**1. 関数定義**

```python
def load_and_preprocess_data(filepaths):
    reviews = []
    labels = []
    
    for filepath in filepaths:
        df = pd.read_csv(filepath)
        reviews.extend(df['text'].tolist())
        labels.extend(df['label'].tolist())

    return reviews, labels
```

- `def load_and_preprocess_data(filepaths)`: 関数名を `load_and_preprocess_data` と定義し, 引数として `filepaths` を受け取ります. 
    - `filepaths`: 読み込むCSVファイルのパスをリスト形式で渡します. 

**2. データ読み込み**

```python
for filepath in filepaths:
    df = pd.read_csv(filepath)
    reviews.extend(df['text'].tolist())
    labels.extend(df['label'].tolist())
```

- `for filepath in filepaths`: `filepaths` リスト内の各ファイルパスに対して処理を繰り返します. 
- `df = pd.read_csv(filepath)`: 指定されたファイルパスからCSVデータを読み込み, DataFrame形式に変換します. 
- `reviews.extend(df['text'].tolist())`: DataFrameの `'text'` 列の値をすべてリスト `reviews` に追加します. 
- `labels.extend(df['label'].tolist())`: DataFrameの `'label'` 列の値をすべてリスト `labels` に追加します. 

**3. 処理結果の返却**

```python
return reviews, labels
```

- 処理が完了したら, `reviews` と `labels` のリストをタプル形式で返します. 

**この関数を実行すると**

- `filepaths` で指定されたCSVファイルから口コミデータと感情ラベルが読み込まれ, リスト形式で返されます. 
- 返されたリストは, Transformerライブラリのデータセット作成などに利用できます. 

In [21]:
# CSVファイルのパスリストを作成
filepaths = [os.path.join('text', f'review_{i}.csv') for i in range(19)]

# データの読み込み
reviews, labels = load_and_preprocess_data(filepaths)

# トークナイザーの初期化
tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-japanese-whole-word-masking')

# データセットの作成
dataset = ReviewDataset(reviews, labels, tokenizer, max_len=500)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)

# 前処理したデータを表示
for i, data in enumerate(data_loader):
    print(f"Review {i}:")
    print(f"Text: {data['review_text'][0]}")
    print(f"Input IDs: {data['input_ids'][0]}")
    print(f"Attention Mask: {data['attention_mask'][0]}")
    print(f"Label: {data['label'][0]}")
    print("="*50)

OSError: cl-tohoku/bert-japanese-whole-word-masking is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

**1. CSVファイルパスリストの作成**

```python
filepaths = [f'review_{i}.csv' for i in range(12)]
```

- `[f'review_{i}.csv' for i in range(12)]`: 0から11までのインデックスを持つリストを作成し, 各要素に `f'review_{i}.csv'` という形式の文字列を代入します. 
    - `f'review_{i}.csv'`: 文字列フォーマットを用いて, `i` を動的に文字列に埋め込みます. 
- 結果的に, `filepaths` リストには以下のファイルパスが含まれます. 
    - `review_0.csv`
    - `review_1.csv`
    - ...
    - `review_10.csv`
    - `review_11.csv`

**2. データの読み込み**

```python
reviews, labels = load_and_preprocess_data(filepaths)
```

- `load_and_preprocess_data` 関数を呼び出し, `filepaths` リストを引数として渡します. 
- 関数から返された口コミデータと感情ラベルを, それぞれ `reviews` と `labels` 変数に格納します. 

**3. トークナイザーの初期化**

```python
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
```

- `BertTokenizer.from_pretrained('bert-base-uncased')`: 事前に学習済みのBERTトークナイザ `bert-base-uncased` をロードし, `tokenizer` 変数に格納します. 
- このトークナイザは, 日本語を含む様々な言語のテキストを, Transformerモデルで利用できる形式に変換するために使用されます. 

**4. データセットの作成**

```python
dataset = ReviewDataset(reviews, labels, tokenizer, max_len=160)
```

- `ReviewDataset` クラスを使って, `reviews`, `labels`, `tokenizer`, `max_len` を引数として `dataset` インスタンスを作成します. 
    - `reviews`: 前処理済みの口コミデータ
    - `labels`: 感情ラベル
    - `tokenizer`: トークナイザ
    - `max_len`: 最大トークン長（160に設定）
- `ReviewDataset` クラスは, Transformerライブラリで利用できる形式のデータセットを生成します. 

**5. データローダーの作成**

```python
data_loader = DataLoader(dataset, batch_size=1, shuffle=False)
```

- `DataLoader` クラスを使って, `dataset` インスタンスを基に `data_loader` インスタンスを作成します. 
    - `batch_size=1`: 1つのバッチに1つのデータのみを含めます. 
    - `shuffle=False`: データの順番をランダムにシャッフルしません. 
- `data_loader` は, ミニバッチ単位でデータを効率的に処理するためのツールです. 

**6. 前処理したデータの表示**

```python
for i, data in enumerate(data_loader):
    print(f"Review {i}:")
    print(f"Text: {data['review_text'][0]}")
    print(f"Input IDs: {data['input_ids'][0]}")
    print(f"Attention Mask: {data['attention_mask'][0]}")
    print(f"Label: {data['label'][0]}")
    print("="*50)
```

- `for i, data in enumerate(data_loader)`: `data_loader` からミニバッチ単位でデータをループ処理します. 
    - `i`: バッチのインデックス
    - `data`: ミニバッチデータ
- 各バッチについて, 以下の情報を表示します. 
    - `Review {i}`: バッチのインデックス
    - `Text`: 元の口コミテキスト
    - `Input IDs`: トークンIDのリスト
    - `Attention Mask`: アテンションマスクのリスト
    - `Label`: 感情ラベル
    - `=` * 50: 区切り線
- この処理により, 前処理された口コミデータがどのようにTransformerモデルに入力されるのかを確認することができます. 

In [15]:
# モデルの定義
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


mps


**1. モデルの定義**

```python
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
```

- `BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)`: 事前に学習済みのBERTモデル `bert-base-uncased` をロードし, `model` 変数に格納します. 
    - `BertForSequenceClassification`: テキスト分類タスクに特化したBERTモデルクラス
    - `'bert-base-uncased'`: 事前に学習済みのモデル名
    - `num_labels=2`: 出力ラベルの数を2つに設定（ポジティブ/ネガティブの2クラス分類）
- このモデルは, 2つの文章を比較したり, 文章の感情を分析したりするようなタスクに適しています. 

**2. デバイス設定**

```python
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

if torch.backends.mps.is_available():
  device = torch.device('mps')
else:
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = model.to(device)
```

- このコードは, モデルを実行するデバイスを設定します. 
    - `torch.device('cuda')`: GPUを利用する場合
    - `torch.device('cpu')`: CPUを利用する場合
- 処理速度向上のため, GPUが利用可能であれば優先的に利用するようにしています. 
- `model.to(device)`: モデルを指定されたデバイスに移動します. 

In [18]:
# モデルの評価
def evaluate_model(model, data_loader, device):
    model.eval()
    predictions = []
    true_labels = []
    probabilities = []

    with torch.no_grad():
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['label'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            probs = torch.nn.functional.softmax(logits, dim=1)

            _, preds = torch.max(logits, dim=1)
            predictions.extend(preds.cpu().numpy())  # CPUに移動してからnumpyに変換
            true_labels.extend(labels.cpu().numpy())  # CPUに移動してからnumpyに変換
            probabilities.extend(probs.cpu().numpy())  # CPUに移動してからnumpyに変換

    return predictions, true_labels, probabilities

# モデルの評価
predictions, true_labels, probabilities = evaluate_model(model, data_loader, device)

# 評価結果の表示
accuracy = sum([1 if pred == true else 0 for pred, true in zip(predictions, true_labels)]) / len(true_labels)
print(f'Accuracy: {accuracy}')
print(classification_report(true_labels, predictions, target_names=['Negative', 'Positive']))

# 各データの予測結果を表示
for i, (review, true_label, pred_label, prob) in enumerate(zip(reviews, true_labels, predictions, probabilities)):
    print(f"Review {i+1}:")
    print(f"Text: {review}")
    print(f"True Label: {'Positive' if true_label == 1 else 'Negative'}")
    print(f"Predicted Label: {'Positive' if pred_label == 1 else 'Negative'}")
    print(f"Probabilities: Positive - {prob[1]:.4f}, Negative - {prob[0]:.4f}")
    print("="*50)

Accuracy: 0.42105263157894735
              precision    recall  f1-score   support

    Negative       0.42      1.00      0.59         8
    Positive       0.00      0.00      0.00        11

    accuracy                           0.42        19
   macro avg       0.21      0.50      0.30        19
weighted avg       0.18      0.42      0.25        19

Review 1:
Text: 鮨 さいとうに4回目の訪問. 今回はランチでディナー同様, 摘みありのコースを注文させて頂きました. タイトルにもありますが, 気付けば2年ぶりの訪問. 食べ歩きをしてると2.3年再訪してないお店が沢山出てきますが, 再訪してもそれほど月日が経ってないように感じるのは私だけでしょうか. . 笑さて, 本題のお料理ですが, 摘み握り共に何を食べても美味しく, 今回も流石の安定感です. 何一つとして気になるものはありません. また, この日は以前よりずっと飲んでみたかった黒龍酒造の『ESHIKOTO』を頂きました. 超入手困難なスパークリングの日本酒です. 当たりがとてもドライで食事の邪魔を一切しないテイスト. 香りはほのかに日本酒を感じさせます. こんな貴重なお酒を入るのはさすが齋藤さんですね. 次回はいつ来れるか分かりませんが, またぜひ伺いたいです. ご馳走様でした. 
True Label: Positive
Predicted Label: Negative
Probabilities: Positive - 0.3823, Negative - 0.6177
Review 2:
Text: 個人的に, 都内では大好きなお店の一つ. 3度目の訪問ですが, さいとうさんは時期が変わっても毎度ツマミも握りも安定感があり素晴らしいですね. この日, 初めて頂く「穴子の白焼き」は今まで食べた事ないくらい身がぷりぷりしており, 特に印象深

/Users/master/.pyenv/versions/3.12.2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/master/.pyenv/versions/3.12.2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/master/.pyenv/versions/3.12.2/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.cap

この出力結果は, BERTモデルを使ったテキスト分類（感情分析）の結果を表示しています. 以下に各部分の説明をします. 

### 評価結果の表示
まず, モデルの全体的な評価結果が表示されます. 

```plaintext
Accuracy: 0.42105263157894735
              precision    recall  f1-score   support

    Negative       0.42      1.00      0.59         8
    Positive       0.00      0.00      0.00        11

    accuracy                           0.42        19
   macro avg       0.21      0.50      0.30        19
weighted avg       0.18      0.42      0.25        19
```

#### 各指標の説明
- **Accuracy**: 全体の正解率. ここでは約42%です. 
- **Precision**: 予測が正解である割合. Negativeクラスでは42%, Positiveクラスでは0%. 
- **Recall**: 実際の正解データに対して正しく予測された割合. Negativeクラスでは100%, Positiveクラスでは0%. 
- **F1-score**: PrecisionとRecallの調和平均. Negativeクラスでは0.59, Positiveクラスでは0.00. 
- **Support**: 各クラスの実際のデータ数. Negativeクラスは8, Positiveクラスは11. 

### 各レビューの予測結果の表示
次に, 各レビューの予測結果が個別に表示されます. 

#### 表示形式の説明
```plaintext
Review {i}:
Text: {review text}
True Label: {'Positive' if true_label == 1 else 'Negative'}
Predicted Label: {'Positive' if pred_label == 1 else 'Negative'}
Probabilities: Positive - {prob[1]:.4f}, Negative - {prob[0]:.4f}
==================================================
```
- **Review {i}**: レビューのインデックス. 
- **Text**: レビューのテキスト. 
- **True Label**: レビューの実際のラベル（PositiveまたはNegative）. 
- **Predicted Label**: モデルによる予測ラベル（PositiveまたはNegative）. 
- **Probabilities**: モデルによる予測確率（PositiveとNegativeの確率）. 

### 各レビューの予測結果の例
```plaintext
Review 1:
Text: 鮨 さいとうに4回目の訪問. ...
True Label: Positive
Predicted Label: Negative
Probabilities: Positive - 0.4394, Negative - 0.5606
==================================================
```
- **Review 1**: インデックス1のレビュー. 
- **Text**: レビューの内容. 
- **True Label**: 実際のラベルはPositive. 
- **Predicted Label**: モデルの予測はNegative. 
- **Probabilities**: Positiveの確率が約0.4394, Negativeの確率が約0.5606. 

### 全体的な結果の解析
- **Accuracy**が42.1%であることから, 全体的にモデルの予測精度は低いです. 
- **Negative**クラスは全て正しく予測されましたが, **Positive**クラスは全て誤った予測となっています. 
- **Precision**や**Recall**が**Positive**クラスで0%となっているため, このモデルは**Positive**クラスを全く識別できていません. 
- **Probabilities**を見ると, 多くのレビューでPositiveとNegativeの確率が近い値になっており, モデルがどちらに分類すべきか確信を持っていないことが伺えます. 

### 改善方法
- **データセットのバランスを改善する**: ポジティブとネガティブのレビューの数を均等にすることで, モデルがどちらのクラスも正しく学習できるようにします. 
- **モデルの再トレーニング**: より多くのデータを使ってモデルを再トレーニングし, 過学習やデータ不足を防ぎます. 
- **ハイパーパラメータのチューニング**: 学習率やバッチサイズなどのハイパーパラメータを調整してモデルの性能を向上させます. 
- **データの前処理の改善**: データの前処理を見直し, モデルがより良い特徴を学習できるようにします. 

In [50]:
import numpy as np

In [51]:
# データの分割
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# トークナイザーの初期化
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# データセットの作成
class ReviewDataset(Dataset):
    def __init__(self, reviews, labels, tokenizer, max_len):
        self.reviews = reviews
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, item):
        review = str(self.reviews[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'review_text': review,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# トレーニングと検証のデータセット
train_dataset = ReviewDataset(
    reviews=train_df.text.to_numpy(),
    labels=train_df.label.to_numpy(),
    tokenizer=tokenizer,
    max_len=500
)

val_dataset = ReviewDataset(
    reviews=val_df.text.to_numpy(),
    labels=val_df.label.to_numpy(),
    tokenizer=tokenizer,
    max_len=500
)

train_data_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# モデルのファインチューニング
from transformers import AdamW, get_linear_schedule_with_warmup

def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
    model = model.train()

    losses = []
    correct_predictions = 0

    for data in data_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_mask'].to(device)
        labels = data['label'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs.logits, dim=1)
        loss = loss_fn(outputs.logits, labels)

        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.float() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()

    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_mask'].to(device)
            labels = data['label'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs.logits, dim=1)
            loss = loss_fn(outputs.logits, labels)

            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.float() / n_examples, np.mean(losses)

# モデルの初期化
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
model = model.to(device)

EPOCHS = 10
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

loss_fn = torch.nn.CrossEntropyLoss().to(device)

# トレーニングループ
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_data_loader,
        loss_fn,
        optimizer,
        device,
        scheduler,
        len(train_df)
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        val_data_loader,
        loss_fn,
        device,
        len(val_df)
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')
    print()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/master/.pyenv/versions/3.12.2/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10
----------
Train loss 1.0304685235023499 accuracy 0.6666666865348816
Val   loss 0.3965409994125366 accuracy 1.0

Epoch 2/10
----------
Train loss 0.40633469820022583 accuracy 0.8888888955116272
Val   loss 0.3124300241470337 accuracy 1.0

Epoch 3/10
----------
Train loss 0.3297441452741623 accuracy 0.8888888955116272
Val   loss 0.19672556221485138 accuracy 1.0

Epoch 4/10
----------
Train loss 0.27607132494449615 accuracy 0.8888888955116272
Val   loss 0.11254560947418213 accuracy 1.0

Epoch 5/10
----------
Train loss 0.22440343722701073 accuracy 0.8888888955116272
Val   loss 0.07830469310283661 accuracy 1.0

Epoch 6/10
----------
Train loss 1.4837076589465141 accuracy 0.8888888955116272
Val   loss 0.07684759050607681 accuracy 1.0

Epoch 7/10
----------
Train loss 0.22204866632819176 accuracy 0.8888888955116272
Val   loss 0.1286872774362564 accuracy 1.0

Epoch 8/10
----------
Train loss 0.2333529144525528 accuracy 0.8888888955116272
Val   loss 0.15087705850601196 accuracy 1.0
